In [1]:
import json
import heapq
import transformers
import subprocess
import vllm
import time
from datetime import datetime
from tqdm import tqdm, trange
from pathlib import Path
from Lean4Gym import *
# from Lean4Python import *
import Lean4Python
import transformers

/home2/wanglei/anaconda3/envs/wanglei/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-07-05 23:24:21,011	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


##### 指定Lean4的项目环境

In [2]:
lean = Lean4Python.Lean4Python("/home2/wanglei/Project")

##### 导入文件

In [3]:
lean.import_file("import Mathlib.Data.Nat.Choose.Sum")
lean.import_file("import Mathlib.Data.Real.Basic")
lean.import_file("import Mathlib.Data.Finset.LocallyFinite")
lean.import_file("open Nat")
lean.import_file("open Finset")
lean.import_file("open BigOperators")

##### 指定lean4的证明目标

In [4]:
init_state = lean.start_goal("{n k : ℕ} (hk : k ≤ n) : choose n (n - k) = choose n k")
init_state.print()

sid: 0
tacticState: n k : ℕ
hk : k ≤ n
⊢ Nat.choose n (n - k) = Nat.choose n k
error: None


##### 执行证明策略

In [5]:
state1 = lean.run_tactic(init_state, "rw[choose_eq_factorial_div_factorial hk]")
state1.print()
state1.isFinish()


sid: 1
tacticState: n k : ℕ
hk : k ≤ n
⊢ Nat.choose n (n - k) = n ! / (k ! * (n - k)!)
error: None


False

In [6]:
state2 = lean.run_tactic(state1, "rw[choose_eq_factorial_div_factorial]")
state2.print()
state2.isFinish()

sid: 2
tacticState: n k : ℕ
hk : k ≤ n
⊢ n ! / ((n - k)! * (n - (n - k))!) = n ! / (k ! * (n - k)!)

n k : ℕ
hk : k ≤ n
⊢ n - k ≤ n
error: None


False

##### 指定模型路径并加载模型

In [7]:
os.environ['TOKENIZERS_PARALLELISM'] = 'false'
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
model_name_or_path = "/home/ssm/models/pythia2.8b_choose"
model = vllm.LLM(
    model=model_name_or_path,
    tensor_parallel_size=1,
    trust_remote_code=True,
    gpu_memory_utilization=0.5,
    dtype='float16'
)
tokenizer = transformers.GPTNeoXTokenizerFast.from_pretrained(model_name_or_path)

OSError: Incorrect path_or_model_id: '/home/ssm/models/pythia2.8b_choose'. Please provide either the path to a local folder or the repo_id of a model on the Hub.

##### 模型产生输出

In [ ]:
#statement = "n m : ℝ hnm : n = 2 * m ⊢ (2 * m + 1) / (m + 1) * (1 / 2) = 2 * (n + 1) / (n + 2) * (1 / 2)"
#statement = "n A B C : ℕ h₀ : n = 3 ^ 17 + 3 ^ 10 h₁ : 11 ∣ n + 1 h₂ : List.Pairwise (fun x x_1 => x ≠ x_1) [A, B, C] h₃ : {A, B, C} ⊂ Finset.Icc 0 9 h₄ : Odd A ∧ Odd C h₅ : ¬3 ∣ B h₆ : digits 10 n = [B, A, B, C, C, A, C, B, A] ⊢ 100 * A + 10 * B + C = 129"
#statement = "b h v : ℝ h₀ : 0 < b ∧ 0 < h ∧ 0 < v h₁ : v = 1 / 3 * (b * h) h₂ : b = 30 h₃ : h = 13 / 2 ⊢ v = 65"
statement = "b h v : ℝ h₀ : 0 < 30 ∧ 0 < 13 / 2 ∧ 0 < 30 * (13 / 2) h₁ : v = 3⁻¹ * (30 * (13 / 2)) h₂ : b = 30 h₃ : h = 13 / 2 ⊢ 3⁻¹ * (30 * (13 / 2)) = 65"

texts, scores = Lean4Python.generate_vllm(Lean4Python._prompt_proofstep(statement), model, tokenizer, 
                              temperatures=[0], num_samples=8, stop=tokenizer.eos_token)
print(statement)
print(texts)
print(scores)

b h v : ℝ h₀ : 0 < 30 ∧ 0 < 13 / 2 ∧ 0 < 30 * (13 / 2) h₁ : v = 3⁻¹ * (30 * (13 / 2)) h₂ : b = 30 h₃ : h = 13 / 2 ⊢ 3⁻¹ * (30 * (13 / 2)) = 65
['norm_num', 'rw [← mul_assoc]', 'norm_num [h]', 'rw [mul_assoc]', 'rw [mul_comm]', 'norm_num1', 'norm_num3', 'simp']
[-0.32128265433311753, -0.39516371421132135, -0.4333076111350757, -0.4612277602291215, -0.5269824257078913, -0.5699598370367311, -0.7771070735776447, -1.1736234289904435]


##### 清空证明目标并重新指定证明目标

In [ ]:
lean.clear_goal()
init_state = lean.start_goal("(n A B C : ℕ) (h₀ : n = 3 ^ 17 + 3 ^ 10) (h₁ : 11 ∣ n + 1)(h₂ : [A, B, C].Pairwise (· ≠ ·)) (h₃ : {A, B, C} ⊂ Finset.Icc 0 9) (h₄ : Odd A ∧ Odd C)(h₅ : ¬3 ∣ B) (h₆ : Nat.digits 10 n = [B, A, B, C, C, A, C, B, A]) : 100 * A + 10 * B + C = 129")
init_state.print()

sid: 0
tacticState: n A B C : ℕ
h₀ : n = 3 ^ 17 + 3 ^ 10
h₁ : 11 ∣ n + 1
h₂ : List.Pairwise (fun x x_1 => x ≠ x_1) [A, B, C]
h₃ : {A, B, C} ⊂ Icc 0 9
h₄ : Odd A ∧ Odd C
h₅ : ¬3 ∣ B
h₆ : sorryAx (List ℕ) true = [B, A, B, C, C, A, C, B, A]
⊢ 100 * A + 10 * B + C = 129
error: None


In [ ]:
statement = init_state.getTacticState()
texts, scores = Lean4Python.generate_vllm(Lean4Python._prompt_proofstep(statement), model, tokenizer, 
                              temperatures=[0], num_samples=8, stop=tokenizer.eos_token)
print(texts)
print(scores)

['norm_num [h₆]\n  norm_num [h₅]\n  norm_num [h₆]', 'norm_num [h₆]\n  norm_num [h₅]\n  norm_num [h₅]', 'norm_num [h₆]\n  norm_num [h₆]\n  norm_num [h₆]', 'norm_num [h₆]\n  norm_num [h₅]\n  norm_num', 'norm_num [h₆]\n  norm_num [h₅]', 'norm_num [h₆]', 'rw h₆ h₅', 'rw h₆']
[-0.35634469091872173, -0.3888711123115078, -0.38951303713236424, -0.41817394055154483, -0.48160612055002466, -0.6919377764893903, -0.8444669029704528, -1.108086042664945]


##### 定理证明搜索

In [ ]:
Lean4Python.search(init_state, lean.leangym, model, tokenizer)

  0%|          | 1/1000000 [00:00<104:25:04,  2.66it/s]

['norm_num [sorryAx] at h₆', 'norm_num [h₆]', 'norm_num [sorryAx]', 'norm_num at h₆', 'rw [sorryAx] at h₆', 'norm_num [A, B, C]', 'rw [← h₆]', 'norm_num1 at h₆', 'simp [h₆]', 'norm_num', 'norm_num1', 'rfl', 'congr 1', 'decide', 'simp', 'congr']


##### 清空环境并重新指定Lean4的项目环境

In [ ]:
lean.clear()
lean = Lean4Python.Lean4Python("/home/ssm/Lean4/Theorem")

In [ ]:
lean.import_file("import Mathlib.Data.Real.Basic")
lean.import_file("import Theorem.example_separate.add_div_two")
lean.import_file("open Nat")
lean.import_file("open Finset")
lean.import_file("open BigOperators")

In [ ]:
print(lean.import_content)

['import Mathlib.Data.Real.Basic', 'import Theorem.example_separate.add_div_two', 'open Nat', 'open Finset', 'open BigOperators']


In [ ]:
init_state = lean.start_goal(": ∑ k in Ico 1 (n + 1), k * choose (n-1) (k-1) = ∑ l in Ico 0 n, (l + 1) * choose (n-1) l")

In [ ]:
lean.tactics = Lean4Python.search(init_state, lean.leangym, model, tokenizer)

  0%|          | 0/1000000 [00:00<?, ?it/s]

['rw [sum_Ico_eq_sub _ (Nat.lt_succ_self _), sum_sub_distrib]', 'rw[sum_Ico_eq_sum_range]', 'rw [sum_Ico_eq_sub _]', 'rw [sum_Ico_eq_sum_range]', 'rw [Ico_choose_eq_Ico_choose_add]', 'rw[Ico_choose_eq_Ico_choose_add]', 'rw[sum_Ico_eq_sub _]', 'rw [← Nat.Ico_choose_eq_Ico_choose_add]', 'rw [← Nat.Ico_choose_eq_Ico]', 'have : 0 < n := by linarith', 'rw [sum_Ico_eq_sub _ (by simp)]', 'have : 1 ≤ n := by linarith', 'rw[sum_congr rfl]', 'rw[sum_Ico_eq]', 'simp', 'cases n']


  0%|          | 1/1000000 [00:00<216:45:38,  1.28it/s]

['simp', 'rw [sum_Ico_eq_sum_range]', "refine' sum_congr rfl fun y _ => _", 'rw[sum_Ico_eq_sum_range]', 'rw [sum_Ico_eq_sub _]', 'rw [sum_range_add_mul]', 'rw [sum_range_add]', 'rw [sum_range_succ]', 'simp [sum_range_add]', 'rw [range_sub_one]', 'rw[sum_range_add]', 'rw[range_sub_one]', 'rw [Nat.sub_one]', 'congr 1', 'rfl', 'congr']


  0%|          | 2/1000000 [00:01<188:09:34,  1.48it/s]

...

['rw [add_mul, mul_assoc]', 'rw [one_mul, mul_assoc]', 'rw [Nat.one_mul]', 'rw [one_mul, add_mul]', 'rw [add_mul, one_mul]', 'rw [add_mul]', 'rw [Nat.add_mul]', 'rw [one_mul, mul_add]', 'rw [one_mul]', 'rw[Nat.add_mul]', 'rw[Nat.one_mul]', 'rw[add_mul]', 'rw [mul_assoc]', 'rw [mul_comm]', 'simp', 'congr 1']
['simp', 'simp (config := { contextual := true }) [add_mul]', 'simp (config := { contextual := true }) [mul_comm]', 'simp [add_mul, mul_assoc]', 'simp (config := { contextual := true })', 'simp [add_mul]', 'simp [mul_comm]', 'intro x hx', 'rw [add_mul]', 'rintro x hx', 'intros x hx', 'intro m hm', 'intro k hk', 'intro i _', 'intro a ha', 'intros']


  0%|          | 38/1000000 [00:21<139:31:03,  1.99it/s]

['rw [add_mul, mul_assoc]', 'rw [Nat.sub_add_cancel hx]', 'rw [Nat.choose_mul hx]', 'rw [Nat.choose_succ_succ]', 'rw [Nat.choose_add_mul hx]', 'rw [Nat.add_mul]', 'rw [Nat.choose_mul_choose]', 'rw [add_mul]', 'rw [Nat.mul_assoc]', 'rw[Nat.add_mul]', 'rw [mul_comm]', 'rw [mul_assoc]', 'rw[add_mul]', 'rw[mul_comm]', 'simp', 'congr 1']
['simp', "refine' sum_congr rfl fun k hk => _", 'rw [sum_range_add]', 'rw [sum_congr rfl]', 'have : 1 ≤ n := by linarith', 'rw[sum_congr rfl]', 'rw[sum_range_add]', 'simp [add_assoc]', 'rw [add_right_comm]', 'rw [sum_range_succ]', 'simp [add_mul]', 'rw [range_sub_one]', 'rw [add_assoc]', 'congr 1', 'rfl', 'congr']


  0%|          | 39/1000000 [00:22<158:39:56,  1.75it/s]

['rw [one_mul, mul_assoc]', 'rw [Nat.one_mul]', 'rw [one_mul]', 'rw[Nat.one_mul]', 'rw [add_assoc]', 'simp [mul_assoc]', 'congr 1', 'simp', 'rw[add_assoc]', 'rw [mul_assoc]', 'rw[one_mul]', 'rw [mul_comm]', 'ac_rfl', 'rfl', 'congr', 'ring']
Theorem has proved!
['rw[sum_Ico_eq_sum_range]', 'simp', "refine' sum_congr rfl fun x hx => _", 'rw [Nat.add_mul, Nat.add_mul]', 'ac_rfl']


##### 将定理写入到Lean4文件中

In [ ]:
lean.write_into_file(theorem_name="test")